In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import *
import os

%matplotlib inline

In [4]:
def calc_mse(img1, img2, axis=2):
    img1 = img1.astype(np.float64)  # convert to float64 to avoid overflow
    img2 = img2.astype(np.float64)  # convert to float64 to avoid overflow

    if axis == 2:
        mse_image = np.linalg.norm(img1 - img2, axis=axis) ** 2
        num_channels = img1.shape[axis]
        mse = np.mean(mse_image) / num_channels
    elif axis == 0:
        mse_image = (img1 - img2) ** 2
        mse = np.mean(mse_image)
    else:
        raise ValueError("axis must be 0 or 2")
    return mse, mse_image


def calc_mse_skimage(img1, img2):
    return mean_squared_error(img1, img2)


def calc_ssim(img1, img2, axis=2):
    mean_ssim, grad, ssim_image = structural_similarity(
        img1, img2, channel_axis=axis, gradient=True, full=True
    )
    return mean_ssim, grad, ssim_image


def calc_psnr(img1, img2, max_pixel_value=255, axis=2):
    mse, _ = calc_mse(img1, img2, axis=axis)
    psnr = 10 * np.log10(max_pixel_value**2 / mse)
    return psnr


def calc_psnr_skimage(img1, img2):
    return peak_signal_noise_ratio(img1, img2)

In [5]:
# gt_dir = "/datassd/KinectStream/panoptic_data/160906_band2_with_ground/"
# dist_dir = "/home/lei/data/pipeline/client_tiled/pipeline_new/test/"

gt_dir = "/datassd/KinectStream/panoptic_data/160906_band2_with_ground/kpculled_views_logID_1"
dist_dir = (
    "/datassd/KinectStream/panoptic_data/160906_band2_with_ground/kpculled_views2_logID_1"
)

In [6]:
# Check RMSE for color images
frame_id = 2
nViews = 10
total_mse = 0.0
total_mse_cv2 = 0.0

height = 0
width = 0
for i in range(0, nViews):
    gt = cv2.imread(
        os.path.join(gt_dir, "color", f"{frame_id}_color_{i}.png"), cv2.IMREAD_COLOR
    )
    # dist = cv2.imread(
    #     os.path.join(dist_dir, f"{frame_id}_dist_color_{i}.png"), cv2.IMREAD_COLOR
    # )
    dist = cv2.imread(
        os.path.join(dist_dir, "color", f"{frame_id}_color_{i}.png"), cv2.IMREAD_COLOR
    )

    height = gt.shape[0]
    width = gt.shape[1]

    mse, mse_image = calc_mse(gt, dist)
    mse_skimage = calc_mse_skimage(gt, dist)
    psnr = calc_psnr(gt, dist)
    psnr_skimage = calc_psnr_skimage(gt, dist)
    ssim, _, _ = calc_ssim(gt, dist)

    diff = cv2.absdiff(dist, gt)
    diff = diff.astype(np.float32)
    diff = cv2.pow(diff, 2, diff)
    mse_cv2 = np.sum(cv2.mean(diff)) / 3

    print(
        f"MSE: {mse}, MSE skimage: {mse_skimage}, MSE cv2: {mse_cv2}, PSNR: {psnr}, PSNR skimage: {psnr_skimage}",
        f"SSIM: {ssim}",
    )
    total_mse += mse
    total_mse_cv2 += mse_cv2

mean_rmse = np.sqrt(total_mse / nViews)
mean_rmse_cv2 = np.sqrt(total_mse_cv2 / nViews)
print(f"Mean RMSE: {mean_rmse}, Mean RMSE cv2: {mean_rmse_cv2}")

/tmp/ipykernel_23276/3900798177.py:30: RuntimeWarning: divide by zero encountered in double_scalars
  psnr = 10 * np.log10(max_pixel_value**2 / mse)
/home/lei/.local/lib/python3.8/site-packages/skimage/metrics/simple_metrics.py:163: RuntimeWarning: divide by zero encountered in double_scalars
  return 10 * np.log10((data_range ** 2) / err)


MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
Mean RMSE: 0.0, Mean RMSE cv2: 0.0


In [7]:
# Check RMSE for depth images
total_mse = 0.0
total_mse_cv2 = 0.0

for i in range(0, nViews):
    gt = cv2.imread(
        os.path.join(gt_dir, "depth", f"{frame_id}_depth_{i}.png"), cv2.IMREAD_UNCHANGED
    )
    # dist = cv2.imread(
    #     os.path.join(dist_dir, f"{frame_id}_dist_depth_{i}.png"), cv2.IMREAD_UNCHANGED
    # )

    dist = cv2.imread(
        os.path.join(dist_dir, "depth", f"{frame_id}_depth_{i}.png"),
        cv2.IMREAD_UNCHANGED,
    )

    mse, mse_image = calc_mse(gt, dist, axis=0)
    mse_skimage = calc_mse_skimage(gt, dist)
    psnr = calc_psnr(gt, dist, max_pixel_value=65535, axis=0)
    psnr_skimage = calc_psnr_skimage(gt, dist)
    ssim, _, _ = calc_ssim(gt, dist, axis=0)

    diff = cv2.absdiff(dist, gt)
    diff = diff.astype(np.float32)
    diff = cv2.pow(diff, 2, diff)
    mse_cv2 = np.sum(cv2.mean(diff))

    print(
        f"MSE: {mse}, MSE skimage: {mse_skimage}, MSE cv2: {mse_cv2}, PSNR: {psnr}, PSNR skimage: {psnr_skimage}",
        f"SSIM: {ssim}",
    )
    total_mse += mse
    total_mse_cv2 += mse_cv2

mean_rmse = np.sqrt(total_mse / nViews)
mean_rmse_cv2 = np.sqrt(total_mse_cv2 / nViews)
print(f"Mean RMSE: {mean_rmse}, Mean RMSE cv2: {mean_rmse_cv2}")

/tmp/ipykernel_23276/3900798177.py:30: RuntimeWarning: divide by zero encountered in double_scalars
  psnr = 10 * np.log10(max_pixel_value**2 / mse)


MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
MSE: 0.0, MSE skimage: 0.0, MSE cv2: 0.0, PSNR: inf, PSNR skimage: inf SSIM: 1.0
Mean RMSE: 0.0, Mean RMSE cv2: 0.0
